In [1]:
!pip install numba opencv-python matplotlib

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pillow cupy
import os
from PIL import Image
import cupy as cp
import numpy as np
import time

def load_images_from_folder(folder_path):
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.png')]
    images = []
    for file in image_files:
        image_path = os.path.join(folder_path, file)
        image = Image.open(image_path).resize((500, 500))  # Garante o tamanho mínimo de 500x500
        images.append(np.array(image))
    return images

def rgb_to_grayscale_cuda(image):
    """
    Converte uma imagem colorida para tons de cinza usando CUDA.
    """
    # Mapeia a imagem para GPU
    d_image = cp.array(image, dtype=cp.float32)

    # Divide a imagem em canais R, G, B
    red_channel = d_image[:, :, 0]
    green_channel = d_image[:, :, 1]
    blue_channel = d_image[:, :, 2]

    # Aplica a fórmula para conversão para tons de cinza
    grayscale = 0.298 * red_channel + 0.587 * green_channel + 0.114 * blue_channel

    # Converte o resultado para uint8 e retorna
    return cp.asnumpy(grayscale.astype(cp.uint8))

def process_images(images):
    """
    Processa um dataset de imagens em paralelo, convertendo para tons de cinza.
    """
    processed_images = []
    for image in images:
        grayscale_image = rgb_to_grayscale_cuda(image)
        processed_images.append(grayscale_image)
    return processed_images

def save_images(images, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for i, image in enumerate(images):
        output_path = os.path.join(output_folder, f'gray_image_{i}.png')
        Image.fromarray(image).save(output_path)


dataset_path = '/content/drive/My Drive/ColabNotebooks/DatasetImgs/imgs/' # Alterar para o caminho do dataset
print("Alterar ")
output_path = '/content/drive/My Drive/ColabNotebooks/DatasetImgs/processed/' # Alterar para o caminho de saída

start_time = time.time()
# Pipeline completo
images = load_images_from_folder(dataset_path)
grayscale_images = process_images(images)
save_images(grayscale_images, output_path)
end_time = time.time()

print(f"Tempo total de execução: {end_time - start_time:.2f} segundos")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
Failed to build cupy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (cupy)
Tempo total de execução: 17.14 segundos
